In [1]:
import pandas as pd
import sys
import sqlite3
import numpy as np
!pip install sklearn --upgrade
!pip install joblib

Requirement already up-to-date: sklearn in c:\users\bocli\anaconda3\lib\site-packages (0.0)


In [2]:
#importing data 
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/Casino.sqlite")
c = con.cursor()

In [3]:
c.execute("SELECT * FROM casinoSW limit 5;")
results= c.fetchall()
print(results)
# for row in c.execute("SELECT * FROM casinoSW"):
#     print(row)

[(0, 2014, 'Arizona', 'Coconino County', ' Tribal Authorized Casino', 'Alters or Cancels Transaction to Avoid CTR Requirement', 1, 35.829692, -111.773728), (1, 2014, 'Arizona', 'Coconino County', ' Tribal Authorized Casino', 'Exchanges Small Bills for Large Bills or Vice Versa', 1, 35.829692, -111.773728), (2, 2014, 'Arizona', 'Coconino County', ' Tribal Authorized Casino', 'Other Structuring', 1, 35.829692, -111.773728), (3, 2014, 'Arizona', 'Coconino County', ' Tribal Authorized Casino', 'Provided Questionable or False Documentation', 1, 35.829692, -111.773728), (4, 2016, 'Arizona', 'Coconino County', ' Tribal Authorized Casino', 'Account Takeover', 1, 35.829692, -111.773728)]


In [4]:
df = pd.read_sql_query("SELECT * FROM casinoSW",  con).dropna()
df

,id,Year,State,Countym,Industry,Suspicious Activity,Count,Lat,Long
0,0,2014,Arizona,Coconino County,Tribal Authorized Casino,Alters or Cancels Transaction to Avoid CTR Req...,1,35.829692,-111.773728
1,1,2014,Arizona,Coconino County,Tribal Authorized Casino,Exchanges Small Bills for Large Bills or Vice ...,1,35.829692,-111.773728
2,2,2014,Arizona,Coconino County,Tribal Authorized Casino,Other Structuring,1,35.829692,-111.773728
3,3,2014,Arizona,Coconino County,Tribal Authorized Casino,Provided Questionable or False Documentation,1,35.829692,-111.773728
4,4,2016,Arizona,Coconino County,Tribal Authorized Casino,Account Takeover,1,35.829692,-111.773728
...,...,...,...,...,...,...,...,...,...
7227,7227,2019,Utah,Summit County,Tribal Authorized Casino,Suspicious use of multiple transaction locations,1,40.872060,-110.968486
7228,7228,2019,Utah,Summit County,Tribal Authorized Casino,Transaction Out of Pattern for Customer(s),1,40.872060,-110.968486
7229,7229,2019,Utah,Summit County,Tribal Authorized Casino,Transaction(s) Below BSA Recordkeeping Threshold,3,40.872060,-110.968486
7230,7230,2019,Utah,Summit County,Tribal Authorized Casino,Transaction(s) Below CTR Threshold,1,40.872060,-110.968486


In [5]:
cleanup_industry = {"Industry": {' Tribal Authorized Casino': 0, ' Card Club':1,
                                 ' State Licensed Casino': 2, ' Other Casino/Card Club': 3}}

In [6]:
df.replace(cleanup_industry, inplace=True)

In [7]:
df

,id,Year,State,Countym,Industry,Suspicious Activity,Count,Lat,Long
0,0,2014,Arizona,Coconino County,0,Alters or Cancels Transaction to Avoid CTR Req...,1,35.829692,-111.773728
1,1,2014,Arizona,Coconino County,0,Exchanges Small Bills for Large Bills or Vice ...,1,35.829692,-111.773728
2,2,2014,Arizona,Coconino County,0,Other Structuring,1,35.829692,-111.773728
3,3,2014,Arizona,Coconino County,0,Provided Questionable or False Documentation,1,35.829692,-111.773728
4,4,2016,Arizona,Coconino County,0,Account Takeover,1,35.829692,-111.773728
...,...,...,...,...,...,...,...,...,...
7227,7227,2019,Utah,Summit County,0,Suspicious use of multiple transaction locations,1,40.872060,-110.968486
7228,7228,2019,Utah,Summit County,0,Transaction Out of Pattern for Customer(s),1,40.872060,-110.968486
7229,7229,2019,Utah,Summit County,0,Transaction(s) Below BSA Recordkeeping Threshold,3,40.872060,-110.968486
7230,7230,2019,Utah,Summit County,0,Transaction(s) Below CTR Threshold,1,40.872060,-110.968486


In [8]:
is_Nevada = df["State"]=="Nevada"
print(is_Nevada)

0       False
1       False
2       False
3       False
4       False
        ...  
7227    False
7228    False
7229    False
7230    False
7231    False
Name: State, Length: 7231, dtype: bool


In [9]:
df_nevada = df[is_Nevada]
print(df_nevada.shape)

(991, 9)


In [15]:
ml_nevada = df_nevada.drop(columns={"Lat", "Long","State","Countym","Suspicious Activity", "id"})

In [16]:
ml_nevada

,Year,Industry,Count
5154,2014,2,1
5155,2014,2,1
5156,2015,2,1
5157,2015,2,1
5158,2015,2,1
...,...,...,...
7205,2019,0,1
7206,2019,0,3
7207,2019,0,1
7208,2019,0,3


In [20]:
# Replacing Count column's strings and "," with int dtypes
ml_nevada["Count"]=pd.Series(ml_nevada["Count"].astype(str))
ml_nevada["Count"] = ml_nevada["Count"].apply(lambda x: float(x.split()[0].replace(',', '')))
ml_nevada["Count"].astype(int)

5154    1
5155    1
5156    1
5157    1
5158    1
       ..
7205    1
7206    3
7207    1
7208    3
7209    3
Name: Count, Length: 991, dtype: int32

In [21]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report as report
from random import randint
from sys import maxsize
import warnings
from sklearn.exceptions import DataConversionWarning,UndefinedMetricWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)
warnings.filterwarnings(action='ignore',category=UndefinedMetricWarning)

#pipeline: 1) scales model 2) reduces dimensionality 3) tests the model

#this function takes an arugent n_features representing the total number of features to keep
#by iterating through n_features=1 to n_features = max_features we will determine the best set of features
"""
    SUMMARY:  Creates a pipeline with the given data preprocessors and model and then tests the model using the pipeline
    
    
    
    IN: n_features = denotes the number of features, n_features, to select from the overall data set
                        there is a total of 40 columns in the input data and we may not want all of that data
                        some of the featuress could be 'noise' and not be useful to predicting the output.
                        We will use Principal Component Analysis, by way of the SelectKBest function and the f_classif function
                        do decide which selection of features is optimal
        
        testmodel = denotes the model we wish to test in this test run.  could be any classification model that supports
                        the functions "fit" and "fit_transform"
                        

            
    OUT:  Returns the overall model precision

"""

def RunTest(n_features,testmodel):
    X= ml_nevada.drop(columns=["Industry"],inplace=False)
    y= ml_nevada["Industry"].values.reshape(-1,1) # same as  y = [ df["koi_disposition"].values ]
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=randint(0,2**32 - 1))
    #testscaler=MinMaxScaler(feature_range=(-1,1),copy=True)
    testscaler=StandardScaler()
    normalizer=Normalizer()
    testpca=SelectKBest(f_classif,k="all")
    mypipeline=Pipeline([('featurescale',testscaler),("mynormalizer",normalizer), ("anova",testpca), ("model",testmodel)])    
    mypipeline.fit(X_train,y_train)
    predictions=mypipeline.predict(X_test)
    return report(y_test,predictions,output_dict=True)
    



precisions={}
models={KNN(n_neighbors=1):"knn",RandomForestClassifier():"rff",SVC():"svc", LogisticRegression():"lin",
       } #chooses all models from dict list beforehand
for model,name in models.items():
    for n_features in range(3,1,-1): #loops through each model and every step of the way it throws out the worst column
        score=RunTest(n_features,model)["weighted avg"]['precision']
        n=n_features
        precisions[float(score)]=(name,n)
answer=max(precisions)
print(answer,str(precisions.get(answer)))

0.8730670022208359 ('knn', 3)


C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify 

In [22]:
NV= pd.read_sql_query("SELECT * FROM casinoSW",  con).dropna()
NV

,id,Year,State,Countym,Industry,Suspicious Activity,Count,Lat,Long
0,0,2014,Arizona,Coconino County,Tribal Authorized Casino,Alters or Cancels Transaction to Avoid CTR Req...,1,35.829692,-111.773728
1,1,2014,Arizona,Coconino County,Tribal Authorized Casino,Exchanges Small Bills for Large Bills or Vice ...,1,35.829692,-111.773728
2,2,2014,Arizona,Coconino County,Tribal Authorized Casino,Other Structuring,1,35.829692,-111.773728
3,3,2014,Arizona,Coconino County,Tribal Authorized Casino,Provided Questionable or False Documentation,1,35.829692,-111.773728
4,4,2016,Arizona,Coconino County,Tribal Authorized Casino,Account Takeover,1,35.829692,-111.773728
...,...,...,...,...,...,...,...,...,...
7227,7227,2019,Utah,Summit County,Tribal Authorized Casino,Suspicious use of multiple transaction locations,1,40.872060,-110.968486
7228,7228,2019,Utah,Summit County,Tribal Authorized Casino,Transaction Out of Pattern for Customer(s),1,40.872060,-110.968486
7229,7229,2019,Utah,Summit County,Tribal Authorized Casino,Transaction(s) Below BSA Recordkeeping Threshold,3,40.872060,-110.968486
7230,7230,2019,Utah,Summit County,Tribal Authorized Casino,Transaction(s) Below CTR Threshold,1,40.872060,-110.968486


In [58]:
is_NV = NV["State"]=="Nevada"
print(is_NV)

0       False
1       False
2       False
3       False
4       False
        ...  
7227    False
7228    False
7229    False
7230    False
7231    False
Name: State, Length: 7231, dtype: bool


In [59]:
df_NV = df[is_NV]
print(df_NV.shape)

(991, 9)


In [68]:
ml_NV = df_NV.drop(columns={"Lat", "Long","Suspicious Activity"})
ml_NV

,id,Year,State,Countym,Industry,Count
5154,5154,2014,Nevada,Carson City,2,1
5155,5155,2014,Nevada,Carson City,2,1
5156,5156,2015,Nevada,Carson City,2,1
5157,5157,2015,Nevada,Carson City,2,1
5158,5158,2015,Nevada,Carson City,2,1
...,...,...,...,...,...,...
7205,7205,2019,Nevada,Lincoln County,0,1
7206,7206,2019,Nevada,Lincoln County,0,3
7207,7207,2019,Nevada,Lincoln County,0,1
7208,7208,2019,Nevada,Lincoln County,0,3


In [64]:
NV_counties= pd.get_dummies(ml_NV["Countym"]).reset_index(level=0)

In [67]:
NV_counties= NV_counties.rename(columns={"index":"id"})
NV_counties

,id,Carson City,Churchill County,Clark County,Douglas County,Elko County,Humboldt County,Lincoln County,Lyon County,Mineral County,Nye County,Storey County,Washoe County
0,5154,1,0,0,0,0,0,0,0,0,0,0,0
1,5155,1,0,0,0,0,0,0,0,0,0,0,0
2,5156,1,0,0,0,0,0,0,0,0,0,0,0
3,5157,1,0,0,0,0,0,0,0,0,0,0,0
4,5158,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,7205,0,0,0,0,0,0,1,0,0,0,0,0
987,7206,0,0,0,0,0,0,1,0,0,0,0,0
988,7207,0,0,0,0,0,0,1,0,0,0,0,0
989,7208,0,0,0,0,0,0,1,0,0,0,0,0


In [76]:
nv_county_df= ml_NV.merge(NV_counties, left_on="id", right_on="id")
nv_county_df = nv_county_df.drop(columns={"State","id","Countym"})

In [77]:
nv_county_df

,Year,Industry,Count,Carson City,Churchill County,Clark County,Douglas County,Elko County,Humboldt County,Lincoln County,Lyon County,Mineral County,Nye County,Storey County,Washoe County
0,2014,2,1,1,0,0,0,0,0,0,0,0,0,0,0
1,2014,2,1,1,0,0,0,0,0,0,0,0,0,0,0
2,2015,2,1,1,0,0,0,0,0,0,0,0,0,0,0
3,2015,2,1,1,0,0,0,0,0,0,0,0,0,0,0
4,2015,2,1,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,2019,0,1,0,0,0,0,0,0,1,0,0,0,0,0
987,2019,0,3,0,0,0,0,0,0,1,0,0,0,0,0
988,2019,0,1,0,0,0,0,0,0,1,0,0,0,0,0
989,2019,0,3,0,0,0,0,0,0,1,0,0,0,0,0


In [79]:
# Replacing Count column's strings and "," with int dtypes
nv_county_df["Count"]=pd.Series(nv_county_df["Count"].astype(str))
nv_county_df["Count"] = nv_county_df["Count"].apply(lambda x: float(x.split()[0].replace(',', '')))
nv_county_df["Count"].astype(int)

0      1
1      1
2      1
3      1
4      1
      ..
986    1
987    3
988    1
989    3
990    3
Name: Count, Length: 991, dtype: int32

In [80]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report as report
from random import randint
from sys import maxsize
import warnings
from sklearn.exceptions import DataConversionWarning,UndefinedMetricWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)
warnings.filterwarnings(action='ignore',category=UndefinedMetricWarning)

#pipeline: 1) scales model 2) reduces dimensionality 3) tests the model

#this function takes an arugent n_features representing the total number of features to keep
#by iterating through n_features=1 to n_features = max_features we will determine the best set of features
"""
    SUMMARY:  Creates a pipeline with the given data preprocessors and model and then tests the model using the pipeline
    
    
    
    IN: n_features = denotes the number of features, n_features, to select from the overall data set
                        there is a total of 40 columns in the input data and we may not want all of that data
                        some of the featuress could be 'noise' and not be useful to predicting the output.
                        We will use Principal Component Analysis, by way of the SelectKBest function and the f_classif function
                        do decide which selection of features is optimal
        
        testmodel = denotes the model we wish to test in this test run.  could be any classification model that supports
                        the functions "fit" and "fit_transform"
                        

            
    OUT:  Returns the overall model precision

"""

def RunTest(n_features,testmodel):
    X= nv_county_df.drop(columns=["Industry"],inplace=False)
    y= nv_county_df["Industry"].values.reshape(-1,1) # same as  y = [ df["koi_disposition"].values ]
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=randint(0,2**32 - 1))
    #testscaler=MinMaxScaler(feature_range=(-1,1),copy=True)
    testscaler=StandardScaler()
    normalizer=Normalizer()
    testpca=SelectKBest(f_classif,k="all")
    mypipeline=Pipeline([('featurescale',testscaler),("mynormalizer",normalizer), ("anova",testpca), ("model",testmodel)])    
    mypipeline.fit(X_train,y_train)
    predictions=mypipeline.predict(X_test)
    return report(y_test,predictions,output_dict=True)
    



precisions={}
models={KNN(n_neighbors=15):"knn",RandomForestClassifier():"rff",SVC():"svc", LogisticRegression():"lin",
       } #chooses all models from dict list beforehand
for model,name in models.items():
    for n_features in range(3,1,-1): #loops through each model and every step of the way it throws out the worst column
        score=RunTest(n_features,model)["weighted avg"]['precision']
        n=n_features
        precisions[float(score)]=(name,n)
answer=max(precisions)
print(answer,str(precisions.get(answer)))

0.9067204301075269 ('rff', 2)


C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will chan